## Supervised Machine Learning - Random Forest Classifier Algoritm (Galaxies vs. QSO)

### Dependencies:

In [1]:
import numpy as np
import pandas as pd

from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import extract

In [2]:
columns = ["obj_ID",'alpha','delta', 'u', 'g', 'r', 'i', 'z', 'run_ID',
       'rerun_ID', 'cam_col', 'field_ID', 'spec_obj_ID', 'redshift',
       'plate', 'MJD', 'fiber_ID']
target = ["class"]

In [3]:
# # Create engine
# engine = create_engine("sqlite:///

In [4]:
# # reflect an existing database into a new model
# Base = automap_base()
# # reflect the tables
# Base.prepare(engine, reflect=True)
# # Save references to each table
# Measurement = Base.classes.measurement
# Station = Base.classes.station

In [5]:
# # Create our session (link) from Python to the DB
# session = Session(engine)

In [6]:
# Load the data
file_path = Path('Resources/qso_galaxy.csv', index=False)
data_df = pd.read_csv(file_path)
# df = df.loc[:, columns].copy()
data_df

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,1.237660e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543780e+18,GALAXY,0.634794,5812,56354,171
1,1.237660e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176010e+19,GALAXY,0.779136,10445,58158,427
2,1.237660e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,1.237660e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030110e+19,GALAXY,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891860e+18,GALAXY,0.116123,6121,56187,842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78401,1.237680e+18,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,7778,301,2,581,1.060000e+19,GALAXY,0.000000,9374,57749,438
78402,1.237680e+18,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,7917,301,1,289,8.586350e+18,GALAXY,0.404895,7626,56934,866
78403,1.237670e+18,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,5314,301,4,308,3.112010e+18,GALAXY,0.143366,2764,54535,74
78404,1.237660e+18,212.268621,46.660365,25.35039,21.63757,19.91386,19.07254,18.62482,3650,301,4,131,7.601080e+18,GALAXY,0.455040,6751,56368,470


In [7]:
data_df.set_index(['spec_obj_ID'], inplace = True)

In [8]:
data_df

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,class,redshift,plate,MJD,fiber_ID
spec_obj_ID,,,,,,,,,,,,,,,,,
6.543780e+18,1.237660e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,GALAXY,0.634794,5812,56354,171
1.176010e+19,1.237660e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,GALAXY,0.779136,10445,58158,427
5.152200e+18,1.237660e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,GALAXY,0.644195,4576,55592,299
1.030110e+19,1.237660e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,GALAXY,0.932346,9149,58039,775
6.891860e+18,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,GALAXY,0.116123,6121,56187,842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.060000e+19,1.237680e+18,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,7778,301,2,581,GALAXY,0.000000,9374,57749,438
8.586350e+18,1.237680e+18,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,7917,301,1,289,GALAXY,0.404895,7626,56934,866
3.112010e+18,1.237670e+18,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,5314,301,4,308,GALAXY,0.143366,2764,54535,74


In [9]:
#drop columns not needed for machine learning 
new_data_df=data_df.drop(['alpha','delta','run_ID','rerun_ID', 'cam_col', 'field_ID', "obj_ID",'plate', 'MJD', 'fiber_ID'], axis = 1 )

In [10]:
new_data_df

,u,g,r,i,z,class,redshift
spec_obj_ID,,,,,,,
6.543780e+18,23.87882,22.27530,20.39501,19.16573,18.79371,GALAXY,0.634794
1.176010e+19,24.77759,22.83188,22.58444,21.16812,21.61427,GALAXY,0.779136
5.152200e+18,25.26307,22.66389,20.60976,19.34857,18.94827,GALAXY,0.644195
1.030110e+19,22.13682,23.77656,21.61162,20.50454,19.25010,GALAXY,0.932346
6.891860e+18,19.43718,17.58028,16.49747,15.97711,15.54461,GALAXY,0.116123
...,...,...,...,...,...,...,...
1.060000e+19,22.16759,22.97586,21.90404,21.30548,20.73569,GALAXY,0.000000
8.586350e+18,22.69118,22.38628,20.45003,19.75759,19.41526,GALAXY,0.404895
3.112010e+18,21.16916,19.26997,18.20428,17.69034,17.35221,GALAXY,0.143366


### Variable engineering

In [11]:
# bin by quartile

frequency = [1, 2, 3]

u_size_bins = [-10000, -6600,-3310, 33]
new_data_df["binned_u"] = pd.cut(new_data_df["u"], u_size_bins, labels = frequency)

g_size_bins = [-10000, -6600,-3310, 33]
new_data_df["binned_g"] = pd.cut(new_data_df["g"], g_size_bins, labels = frequency)

r_size_bins = [9,16,23, 30]
new_data_df["binned_r"] = pd.cut(new_data_df["r"], r_size_bins, labels = frequency)

i_size_bins = [9,17,25, 33]
new_data_df["binned_i"] = pd.cut(new_data_df["i"], i_size_bins, labels = frequency)

z_size_bins = [-10000, -6600,-3310, 30]
new_data_df["binned_z"] = pd.cut(new_data_df["z"], z_size_bins, labels = frequency)

new_data_df.sample(30)

,u,g,r,i,z,class,redshift,binned_u,binned_g,binned_r,binned_i,binned_z
spec_obj_ID,,,,,,,,,,,,
6.824120e+18,22.90935,20.38962,18.79656,18.20639,17.85526,GALAXY,0.318200,3,3,2,2,3
6.948330e+17,19.59636,17.42952,16.42762,15.98191,15.61058,GALAXY,0.107722,3,3,2,1,3
9.958700e+18,21.25550,21.08849,20.97834,20.77301,20.56856,QSO,0.766414,3,3,2,2,3
5.216490e+18,25.09531,22.66883,21.02424,20.04822,19.70646,GALAXY,0.477728,3,3,2,2,3
9.869850e+18,23.98484,23.62012,22.93762,21.93447,22.58329,GALAXY,0.839718,3,3,2,2,3
4.465390e+18,21.44057,21.33299,20.54453,20.52523,20.37429,GALAXY,0.410659,3,3,2,2,3
4.520000e+18,25.31411,20.93341,19.18496,18.50832,18.07955,GALAXY,0.311890,3,3,2,2,3
6.939180e+18,23.57507,21.81000,20.22061,19.40062,18.84428,GALAXY,0.479359,3,3,2,2,3
6.779300e+18,23.57257,22.26296,20.74163,19.61807,19.15872,GALAXY,0.555305,3,3,2,2,3


In [12]:
new_data_df.iloc[new_data_df.values==np.inf]
new_data_df =new_data_df[~new_data_df.isin([np.nan, np.inf, -np.inf]).any(1)]

### Test and Train Data

In [13]:
# Create our features
X = new_data_df.drop('class', axis=1)

# Create our target
y = new_data_df['class']

Counter(y)

Counter({'GALAXY': 59445, 'QSO': 18961})

In [14]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,stratify=y)
Counter(y_train)

Counter({'QSO': 14221, 'GALAXY': 44583})

### Random Oversampling

In [15]:
# Resample the training data with the RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'QSO': 44583, 'GALAXY': 44583})

### Random Forest Classifier Algorithm (Galaxies vs. QSO) - Oversampling

In [16]:
# Train the Random Forest Classifier Model using the resampled data
model = RandomForestClassifier(n_estimators=100)
model.fit(X_resampled, y_resampled)

RandomForestClassifier()

In [17]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9611639743871465

In [18]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

     GALAXY       0.98      0.99      0.94      0.98      0.96      0.93     14862
        QSO       0.95      0.94      0.99      0.94      0.96      0.92      4740

avg / total       0.97      0.97      0.95      0.97      0.96      0.93     19602



In [19]:
# Display the confusion matrix
c= confusion_matrix(y_test,y_pred)
cmros = pd.DataFrame(c, index=["Actual - Galaxy", "Actual - Star"], columns=["Predicted - Galaxy","Predicted - Star"])
cmros

,Predicted - Galaxy,Predicted - Star
Actual - Galaxy,14642,220
Actual - Star,298,4442


### Random Undersampling

In [20]:
# Resample the training data with the RandomUnderSampler
rus = RandomUnderSampler(random_state=1)
X_resampled1, y_resampled1 = rus.fit_resample(X_train, y_train)
Counter(y_resampled1)

Counter({'GALAXY': 14221, 'QSO': 14221})

### Random Forest Classifier Algorithm (Galaxies vs. QSO) - Undersampling

In [21]:
# Train the Random Forest Classifier Model using the resampled data
model = RandomForestClassifier(n_estimators=100)
model.fit(X_resampled1, y_resampled1)

RandomForestClassifier()

In [22]:
# Calculated the balanced accuracy score
y_pred2 = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred2)

0.9635488264182377

In [23]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred2))

                   pre       rec       spe        f1       geo       iba       sup

     GALAXY       0.98      0.97      0.95      0.98      0.96      0.93     14862
        QSO       0.92      0.95      0.97      0.94      0.96      0.93      4740

avg / total       0.97      0.97      0.96      0.97      0.96      0.93     19602



In [24]:
# Display the confusion matrix
c= confusion_matrix(y_test,y_pred2)
cmrus = pd.DataFrame(c, index=["Actual - Galaxy",b"Actual - Star"], columns=["Predicted - Galaxy","Predicted - Star"])
cmrus

,Predicted - Galaxy,Predicted - Star
Actual - Galaxy,14484,378
b'Actual - Star',225,4515
